# Zadanie 1
## Metoda Gaussa-Jordana

In [2]:
import numpy as np
import time

#### Naive implementation

In [3]:
def gauss_jordan_elimination(matrix, vector):
    n = matrix.shape[0]

    for i in range(n):
        pivot = matrix[i, i]
        for row in range(i+1, n):
            coefficient = matrix[row, i] / pivot
            for column in range(i, n):
                matrix[row, column] -= coefficient * matrix[i, column]
            vector[row] -= coefficient * vector[i]

    for row in range(n-1, -1, -1):
        for column in range(n-1, row, -1):
            vector[row] -= matrix[row, column] * vector[column]
        vector[row] = vector[row] / matrix[row, row]
    return vector

#### Partial pivoting

In [4]:
def partial_pivoting(matrix, vector):
    n = matrix.shape[0]
    row_index = [i for i in range(n)]

    for i in range(n):
        # Partial pivoting
        p = i
        for k in range(i+1, n):
            if abs(matrix[row_index[k], i]) > abs(matrix[row_index[p], i]):
                p = k
        row_index[i], row_index[p] = row_index[p], row_index[i]

        pivot = matrix[row_index[i], i]
        for row in range(i+1, n):
            coefficient = matrix[row_index[row], i] / pivot
            for column in range(i, n):
                matrix[row_index[row], column] -= coefficient * matrix[row_index[i], column]
            vector[row_index[row]] -= coefficient * vector[row_index[i]]

    for row in range(n-1, -1, -1):
        for column in range(n-1, row, -1):
            vector[row_index[row]] -= matrix[row_index[row], column] * vector[row_index[column]]
        vector[row_index[row]] = vector[row_index[row]] / matrix[row_index[row], row]
    return [vector[i] for i in row_index]

#### Simple test

x + y = 3
2x + y = 4
x + y + z = 5

In [5]:
A = np.matrix([
    [1, 1, 0],
    [2, 1, 0],
    [1, 1, 1]
], dtype=np.double)
Y = np.array([3,
              4,
              5], dtype=np.double)

# result = gauss_jordan_elimination(A, Y)
result = partial_pivoting(A, Y)

for i in range(len(result)):
    print(f"{chr(ord('x') + i)} = {result[i]}")

x = 1.0
y = 2.0
z = 2.0


## Correctness check
####

In [6]:
def float_eq(float1, float2, eps=10**-5):
    return abs(float1 - float2) < eps


class Corectness_tester:
    def __init__(self, solving_function, number_of_tests, array_size, number_constrains=(-10**5, 10**5), tester_function=np.linalg.solve):
        self.solving_function = solving_function
        self.tester_function = tester_function
        self.tests_no = number_of_tests
        self.size = array_size
        self.min_val, self.max_val = number_constrains

    def __generate_arrays(self):
        self.arrays_A = [np.random.uniform(self.min_val, self.max_val, (self.size, self.size)) for _ in range(self.tests_no)]
        self.vectors_Y = [np.random.uniform(self.min_val, self.max_val, self.size) for _ in range(self.tests_no)]

    def run(self):
        self.__generate_arrays()
        for (array, vector) in zip(self.arrays_A, self.vectors_Y):
            arr_cpy, vector_cpy = array.copy(), vector.copy()
            result1 = self.solving_function(array, vector)
            result2 = self.tester_function(arr_cpy, vector_cpy)

            for i in range(len(result1)):
                if not float_eq(result1[i], result2[i]):
                    print(f'Results differ at index {i}: {result1[i]} instead of {result2[i]}')
                    return False
        return True

#### Tests and results

In [14]:
test_spec = [(10, 10), (20, 5), (30, 4), (50, 2), (100, 1)]

for function in (gauss_jordan_elimination, partial_pivoting):
    print(f'Function: {function.__name__}')
    i = 1
    for (array_size, number_of_tests) in test_spec:
        print(f"Test {i}: ", end='')
        if Corectness_tester(solving_function=function, number_of_tests=number_of_tests, array_size=array_size).run():
            print("PASSED")
        else:
            print("FAILED")
            break
        i += 1
    print("")

Function: gauss_jordan_elimination
Test 1: PASSED
Test 2: PASSED
Test 3: PASSED
Test 4: PASSED
Test 5: PASSED

Function: partial_pivoting
Test 1: PASSED
Test 2: PASSED
Test 3: PASSED
Test 4: PASSED
Test 5: PASSED



## Efficiency compiration

In [11]:
class Performance_tester:
    def __init__(self, solving_function, number_of_tests, array_size, number_constrains=(-10**5, 10**5)):
        self.solving_function = solving_function
        self.tests_no = number_of_tests
        self.size = array_size
        self.min_val, self.max_val = number_constrains

    def __generate_arrays(self):
        self.arrays_A = [np.random.uniform(self.min_val, self.max_val, (self.size, self.size)) for _ in range(self.tests_no)]
        self.vectors_Y = [np.random.uniform(self.min_val, self.max_val, self.size) for _ in range(self.tests_no)]

    def __solve(self):
        for (a, y) in zip(self.arrays_A, self.vectors_Y):
            self.solving_function(a, y)

    def run(self):
        self.__generate_arrays()
        print(f"Function '{self.solving_function.__name__}', N={self.size}, {number_of_tests} {'tests' if number_of_tests > 1 else 'test'}")

        start = time.time()
        self.__solve()
        end = time.time()
        print("Time elapsed: %.2fs" % (end - start))

#### Tests and results

In [17]:
test_spec = [(100, 5), (200, 3), (300, 1)]
# test_spec = [(550, 2), (600, 1), (700, 1), (800, 1)]

for (array_size, number_of_tests) in test_spec:
    for function in (np.linalg.solve, gauss_jordan_elimination, partial_pivoting):
        Performance_tester(solving_function=function, number_of_tests=number_of_tests, array_size=array_size).run()
        print('')

Function 'solve', N=100, 5 tests
Time elapsed: 0.02s

Function 'gauss_jordan_elimination', N=100, 5 tests
Time elapsed: 0.80s

Function 'partial_pivoting', N=100, 5 tests
Time elapsed: 0.85s

Function 'solve', N=200, 3 tests
Time elapsed: 0.00s

Function 'gauss_jordan_elimination', N=200, 3 tests
Time elapsed: 3.66s

Function 'partial_pivoting', N=200, 3 tests
Time elapsed: 3.88s

Function 'solve', N=300, 1 test
Time elapsed: 0.00s

Function 'gauss_jordan_elimination', N=300, 1 test
Time elapsed: 4.09s

Function 'partial_pivoting', N=300, 1 test
Time elapsed: 4.38s



#### Results on my computer:
Function 'solve', N=100, 5 tests
Time elapsed: 0.02s

Function 'gauss_jordan_elimination', N=100, 5 tests
Time elapsed: 0.80s

Function 'partial_pivoting', N=100, 5 tests
Time elapsed: 0.85s

Function 'solve', N=200, 3 tests
Time elapsed: 0.00s

Function 'gauss_jordan_elimination', N=200, 3 tests
Time elapsed: 3.66s

Function 'partial_pivoting', N=200, 3 tests
Time elapsed: 3.88s

Function 'solve', N=300, 1 test
Time elapsed: 0.00s

Function 'gauss_jordan_elimination', N=300, 1 test
Time elapsed: 4.09s

Function 'partial_pivoting', N=300, 1 test
Time elapsed: 4.38s